In [11]:
print(os.listdir(midi_folder))


['041850225ecaf69a0a30dcd754c62b9a.mid', '065b3f55211e23ea3b1a8072fed320f1.mid', '074795cc57bd82ddd192c1a628a0cee1.mid', '1361e390c33e871b4031a7b165fb8eb8.mid', '153be3742e4ded39d355c04d16de6ced.mid', '15ef3a3a75da60290064c64f507b1ae4.mid', '183ffc7f1a8e6a15e6b36d2bb0efc066.mid', '194b9bb260c257cade1fde7d320d0b9e.mid', '1a7ee3885d7efe83289b31f3c040d756.mid', '1e83b692169848bea70edf5112b0a613.mid', '2a8d611e3f91f064290fc6bb9912c984.mid', '306658bd4f053b02dfd521d5d85d023c.mid', '35421e33da3e9431fc918bda1b27c3ac.mid', '361e811fe3eaac87089d3eee64aebb72.mid', '3665c23b01f3bf656cea2acf662a9f09.mid', '38898446ec0f906d68aa6f2e7f9f509c.mid', '3cab726f1f2d5266e447b34d853f0488.mid', '45473ae79c1ad9073605507881ed8a4c.mid', '47ddfb5720bea99a03f6a937e10e9be6.mid', '6818144d3f5a78fcc589574342464571.mid', '69d75617e7afdc263c1d66cdd2e83a75.mid', '6f565298a2ed6300b83f4b4c0148485a.mid', '6ffd32801401be86c072da1f8c4b5ed7.mid', '7d7355210387d3627501c54c4ff1c582.mid', '7f405633e99cddb7c4dd104b379ae49f.mid',

In [15]:
import os
from music21 import converter, instrument, note, chord

# Set your folder path here
midi_folder = 'classicmusic'
notes = []

# Go through each MIDI file
for file in os.listdir(midi_folder):
    if file.endswith(".mid") or file.endswith(".midi"):
        file_path = os.path.join(midi_folder, file)
        print(f"🎵 Parsing {file_path}")

        try:
            midi = converter.parse(file_path)

            # Handle instruments if available
            parts = instrument.partitionByInstrument(midi)
            if parts:  # If instrument parts exist
                elements = parts.parts[0].recurse()
            else:  # No instruments, use full score
                elements = midi.recurse()

            # Extract notes and chords
            for element in elements:
                if isinstance(element, note.Note):
                    notes.append(str(element.pitch))
                elif isinstance(element, chord.Chord):
                    notes.append('.'.join(str(n) for n in element.normalOrder))

            print(f"✅ Collected from {file}: {len(notes)} notes/chords so far")

        except Exception as e:
            print(f"❌ Failed to parse {file}: {e}")

print("🎯 Total notes/chords collected:", len(notes))


🎵 Parsing classicmusic\041850225ecaf69a0a30dcd754c62b9a.mid
✅ Collected from 041850225ecaf69a0a30dcd754c62b9a.mid: 0 notes/chords so far
🎵 Parsing classicmusic\065b3f55211e23ea3b1a8072fed320f1.mid
✅ Collected from 065b3f55211e23ea3b1a8072fed320f1.mid: 0 notes/chords so far
🎵 Parsing classicmusic\074795cc57bd82ddd192c1a628a0cee1.mid
✅ Collected from 074795cc57bd82ddd192c1a628a0cee1.mid: 0 notes/chords so far
🎵 Parsing classicmusic\1361e390c33e871b4031a7b165fb8eb8.mid
✅ Collected from 1361e390c33e871b4031a7b165fb8eb8.mid: 0 notes/chords so far
🎵 Parsing classicmusic\153be3742e4ded39d355c04d16de6ced.mid
✅ Collected from 153be3742e4ded39d355c04d16de6ced.mid: 0 notes/chords so far
🎵 Parsing classicmusic\15ef3a3a75da60290064c64f507b1ae4.mid
✅ Collected from 15ef3a3a75da60290064c64f507b1ae4.mid: 0 notes/chords so far
🎵 Parsing classicmusic\183ffc7f1a8e6a15e6b36d2bb0efc066.mid
✅ Collected from 183ffc7f1a8e6a15e6b36d2bb0efc066.mid: 0 notes/chords so far
🎵 Parsing classicmusic\194b9bb260c257cade

In [2]:
pitchnames = sorted(set(notes))
n_vocab = len(pitchnames)
print(f"Total unique notes: {n_vocab}")


Total unique notes: 0


In [3]:
note_to_int = dict((note, number) for number, note in enumerate(pitchnames))

In [4]:
sequence_length = 100
network_input = []
network_output = []

for i in range(len(notes) - sequence_length):
    seq_in = notes[i:i + sequence_length]
    seq_out = notes[i + sequence_length]
    network_input.append([note_to_int[n] for n in seq_in])
    network_output.append(note_to_int[seq_out])


In [5]:
import numpy as np
n_patterns = len(network_input)

network_input = np.reshape(network_input, (n_patterns, sequence_length, 1))
network_input = network_input / float(n_vocab)

network_output = np.array(network_output)

In [6]:
import pandas as pd
print(pd.__version__)


2.2.3


In [7]:
from keras.models import Sequential
from keras.layers import LSTM, Dropout, Dense, Activation

model = Sequential()
model.add(LSTM(512, input_shape=(network_input.shape[1], network_input.shape[2]), return_sequences=True))
model.add(Dropout(0.3))
model.add(LSTM(512))
model.add(Dense(256))
model.add(Dropout(0.3))
model.add(Dense(n_vocab))
model.add(Activation('softmax'))
model.compile(loss='sparse_categorical_crossentropy', optimizer='adam')
model.summary()

C:\Users\Devarsh\AppData\Roaming\Python\Python312\site-packages\keras\src\layers\rnn\rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ lstm (LSTM)                     │ (None, 100, 512)       │     1,052,672 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 100, 512)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_1 (LSTM)                   │ (None, 512)            │     2,099,200 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 256)            │       131,328 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 256)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 0)              │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ activation (Activation)         │ (None, 0)              │             0 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 3,283,200 (12.52 MB)

 Trainable params: 3,283,200 (12.52 MB)

 Non-trainable params: 0 (0.00 B)

In [8]:
# Ensure notes are populated
if len(notes) == 0:
	print("⚠️ The notes list is empty. Please check the MIDI folder and parsing logic.")
else:
	# Recreate network_input and network_output
	network_input = []
	network_output = []

	for i in range(len(notes) - sequence_length):
		seq_in = notes[i:i + sequence_length]
		seq_out = notes[i + sequence_length]
		network_input.append([note_to_int[n] for n in seq_in])
		network_output.append(note_to_int[seq_out])

	# Reshape and normalize network_input
	n_patterns = len(network_input)
	network_input = np.reshape(network_input, (n_patterns, sequence_length, 1))
	network_input = network_input / float(n_vocab)

	# Convert network_output to a numpy array
	network_output = np.array(network_output)

	# Train the model
	mymodel = model.fit(network_input, network_output, epochs=10, batch_size=60)


⚠️ The notes list is empty. Please check the MIDI folder and parsing logic.


In [9]:
import random
import numpy as np
start = np.random.randint(0, len(network_input)-1)
pattern = network_input[start]
pattern = pattern.reshape(1, sequence_length, 1)


ValueError: high <= 0